In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio

In [ ]:
import os
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt

import seaborn as sns
import cv2
import glob

print("Tensorflow version " + tf.__version__)

Tensorflow version 2.11.0


In [ ]:
import warnings
import sys
from tqdm import tqdm

import random
from sklearn.model_selection import train_test_split
%matplotlib inline

from urllib.parse import urlparse
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, MaxPool2D, BatchNormalization, Dropout, Activation, Concatenate

from tensorflow.keras import backend as K, Input, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow.keras as keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, Callback , ModelCheckpoint
from tensorflow.keras.metrics import Accuracy,binary_crossentropy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
#from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
csv_path = "/content/drive/MyDrive/Genetic_Algorithm_CNN_Project/Folds.csv"
dir= "/content/drive/MyDrive/Genetic_Algorithm_CNN_Project/BreaKHis_v1/"

In [ ]:
def dataset(csv_path, classes, dir_path):
  df = pd.read_csv(csv_path)

  df["label"] = df['filename'].apply(lambda x: x.split("/")[3])
  df['class'] = df['label'].apply(lambda x: 0 if x =='benign' else 1)

  train_df = df[df['grp'] == 'train']
  test_df = df[df['grp'] == 'test']

  train_df = train_df.sample(frac = 1).reset_index()
  test_df = test_df.sample(frac = 1).reset_index()

  train_df['filename'] = train_df['filename'].apply(lambda x: os.path.join(dir_path, x))
  test_df['filename'] = train_df['filename'].apply(lambda x: os.path.join(dir_path, x))

  return train_df, test_df

In [ ]:
classes = ["benign","malign"]
train_df, test_df = dataset(csv_path, classes, dir)

In [ ]:
train_df

,index,fold,mag,grp,filename,label,class
0,23812,4,40,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,benign,0
1,9030,2,200,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,benign,0
2,3531,1,200,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,malignant,1
3,28565,4,200,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,malignant,1
4,18227,3,400,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,malignant,1
...,...,...,...,...,...,...,...
25875,34066,5,100,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,malignant,1
25876,4898,1,40,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,malignant,1
25877,9577,2,40,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,benign,0
25878,32955,5,400,train,/content/drive/MyDrive/Genetic_Algorithm_CNN_P...,benign,0


In [ ]:
class CustomImageGenerator(keras.utils.Sequence):
    def __init__(self, image_paths, targets, batch_size, image_size):
        self.image_paths = image_paths
        self.targets = targets
        self.batch_size = batch_size
        self.image_size = image_size
        self.num_classes = len(np.unique(targets))
        self.indexes = np.arange(len(self.image_paths))
        self.on_epoch_end()
    
    def __len__(self):
        return int(np.ceil(len(self.image_paths) / float(self.batch_size)))
    
    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_image_paths = [self.image_paths[i] for i in batch_indexes]
        batch_targets = [self.targets[i] for i in batch_indexes]
        batch_images = np.zeros((len(batch_image_paths), *self.image_size, 3))
        for i, path in enumerate(batch_image_paths):
            img = keras.preprocessing.image.load_img(path, target_size=self.image_size)
            img = keras.preprocessing.image.img_to_array(img)
            img /= 255.0
            batch_images[i] = img
        batch_targets = keras.utils.to_categorical(batch_targets, num_classes=self.num_classes)
        return batch_images, batch_targets
    
    def on_epoch_end(self):
        np.random.shuffle(self.indexes)


In [ ]:
traingen = CustomImageGenerator(train_df['filename'].values, train_df['class'].values, batch_size=32, image_size= (256,256))
valgen = CustomImageGenerator(test_df['filename'].values, test_df['class'].values, batch_size=32, image_size= (256,256))

In [ ]:
X, y = next(iter(traingen))

In [ ]:
X.shape

(32, 256, 256, 3)

In [ ]:
y.shape

(32, 2)

In [ ]:
X_val, y_val = next(iter(valgen))

In [ ]:
X_val.shape

(32, 256, 256, 3)

In [ ]:
y_val.shape

(32, 2)

In [ ]:
# input_layer = tf.keras.Input(shape=(256,256, 3))

# x = Conv2D(64, (2,2), activation='relu')(input_layer)
# x = MaxPooling2D((2,2))(x)
# x = Conv2D(32, (2,2), activation='relu')(x)
# x = MaxPooling2D((2,2))(x)
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# x = Dropout(0.4)(x)

# output_layer = Dense(1, activation='sigmoid')(x)

# fuc_model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
# fuc_model.summary()

In [ ]:
# fuc_model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
# fuc_model.fit(traingen,
#                   batch_size = 32, 
#                   epochs=30,
#                   validation_data = valgen, verbose=1)

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libcudnn8 is already the newest version (8.4.1.50-1+cuda11.6).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


In [ ]:
import random
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.optimizers import Adam, RMSprop, SGD

In [ ]:
parameter_values = {
    'filters': [32, 64, 128],
    'kernel_size': [3, 5, 7],
    'dropout': [0, 0.25, 0.5],
    'optimizer': [Adam, RMSprop, SGD],
    'batch_size': [16, 32, 64],
    'epochs': [10, 20, 30],
    'activation': ['relu', 'sigmoid', 'tanh']
}

In [ ]:
def fitness_function(parameters):
    
    model = Sequential()
    model.add(Conv2D(parameters['filters'], parameters['kernel_size'], activation=parameters['activation'], input_shape=(256, 256, 3)))
    model.add(MaxPooling2D())
    model.add(Conv2D(parameters['filters'], parameters['kernel_size'], activation=parameters['activation']))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dense(128, activation=parameters['activation']))
    model.add(Dropout(parameters['dropout']))
    model.add(Dense(1, activation='sigmoid'))
    
    
    optimizer = parameters['optimizer'](lr=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    
    history = model.fit(traingen, epochs=parameters['epochs'], validation_data=valgen)
    validation_acc = history.history['val_acc'][-1]
    
    
    return validation_acc


In [ ]:
def mutation(parameters, mutation_rate):
    # Mutate a single hyperparameter with a certain probability
    if random.random() < mutation_rate:
        param = random.choice(list(parameters.keys()))
        if param == 'dropout':
            parameters[param] = random.uniform(0, 0.5)
        else:
            parameters[param] = random.choice(parameter_values[param])
    return parameters

def crossover(parent1, parent2):
    # Perform crossover between two parents to create a new child
    child = {}
    for param in parent1:
        if random.random() < 0.5:
            child[param] = parent1[param]
        else:
            child[param] = parent2[param]
    return child

def selection(population, num_parents):
    # Select the fittest individuals from the population
    return sorted(population, key=lambda x: x[1], reverse=True)[:num_parents]


In [ ]:
import csv

# Define the main genetic algorithm function
def genetic_algorithm(parameter_values, population_size, num_parents, mutation_rate, generations, save_file):
    # Initialize the population with random hyperparameters
    population = [{'filters': random.choice(parameter_values['filters']),
                   'kernel_size': random.choice(parameter_values['kernel_size']),
                   'dropout': random.uniform(0, 0.5),
                   'optimizer': random.choice(parameter_values['optimizer']),
                   'batch_size': random.choice(parameter_values['batch_size']),
                   'epochs': random.choice(parameter_values['epochs']),
                   'activation': random.choice(parameter_values['activation'])} for i in range(population_size)]
    
    # Open a CSV file to write the results
    with open(save_file, mode='w') as result_file:
        fieldnames = ['Generation', 'Epochs', 'Batch Size', 'Optimizer', 'Accuracy']
        writer = csv.DictWriter(result_file, fieldnames=fieldnames)
        writer.writeheader()

        # Iterate over generations
        for g in range(generations):
            # Evaluate the fitness of each individual in the population
            fitness_scores = [(parameters, fitness_function(parameters)) for parameters in population]

            # Select the fittest individuals from the population
            parents = selection(fitness_scores, num_parents)

            # Create the next generation of the population
            new_population = parents[:]
            while len(new_population) < population_size:
                parent1, parent2 = random.sample(parents, 2)
                child = crossover(parent1[0], parent2[0])
                child = mutation(child, mutation_rate)
                new_population.append(child)

            # Replace the old population with the new one
            population = new_population[:]

            # Print the best fitness score for this generation
            best_fitness = max(fitness_scores, key=lambda x: x[1])[1]
            print(f"Generation {g}: Best validation accuracy = {best_fitness}")
            
            # Save the best hyperparameters and their corresponding validation accuracy to the file
            best_parameters = max(fitness_scores, key=lambda x: x[1])[0]
            writer.writerow({'Generation': g, 'Epochs': best_parameters['epochs'], 'Batch Size': best_parameters['batch_size'], 'Optimizer': best_parameters['optimizer'], 'Accuracy': best_fitness})
        
    # Return the best individual from the final population
    best_parameters = max(fitness_scores, key=lambda x: x[1])[0]
    return best_parameters


In [ ]:
best_parameters = genetic_algorithm(parameter_values, population_size=10, num_parents=4, mutation_rate=0.1, generations=5, save_file='results.csv')

/usr/local/lib/python3.9/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/30
  9/809 [..............................] - ETA: 3:58:34 - loss: 0.7070 - accuracy: 0.5000

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/IPython/core/interactiveshell.py", line 3326, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-22-f852ea70d016>", line 1, in <module>
    best_parameters = genetic_algorithm(parameter_values, population_size=10, num_parents=4, mutation_rate=0.1, generations=5, save_file='results.csv')
  File "<ipython-input-21-83619c941356>", line 23, in genetic_algorithm
    fitness_scores = [(parameters, fitness_function(parameters)) for parameters in population]
  File "<ipython-input-21-83619c941356>", line 23, in <listcomp>
    fitness_scores = [(parameters, fitness_function(parameters)) for parameters in population]
  File "<ipython-input-19-9fb20affd8d7>", line 18, in fitness_function
    history = model.fit(traingen, epochs=parameters['epochs'], validation_data=valgen)
  File "/usr/local/lib/python3.9/dist-packages/keras/utils/traceback_utils.py", line 65, in error_handler
 

KeyboardInterrupt: ignored

In [ ]:
# images[0]